In [10]:
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils
from keras.layers import Input
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
from PIL import Image
from numpy import *
# SKLEARN
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [2]:
# train.iloc[,0]  is band_1 with 5625 float elements
# train.iloc[,1] is band_2 with 5625 float elements
# train.iloc[,2] is id
# train.iloc[,3] is "inc_angle"
# train.iloc[,4] is "is_iceberg"
#print train.iloc[:,0]
df_train = pd.read_json("train.json")
df_train = pd.DataFrame(df_train)
numpydf_train = df_train.values
df_test = pd.read_json("test.json")
df_test = pd.DataFrame(df_test)
numpydf_test = df_test.values

In [3]:
band1_train = df_train['band_1'].values
band2_train = df_train['band_2'].values
inc_angle_train = df_train['inc_angle'].values
print band1_train.shape
print band2_train.shape
print inc_angle_train.shape

y = df_train['is_iceberg'].values
print y

(1604,)
(1604,)
(1604,)
[0 0 1 ..., 0 0 0]


In [4]:
images_train=[]
for cnt,img in enumerate(band1_train):
    images_train.append([img,band2_train[cnt]])
npimages_train = np.array(images_train)
npimages_train=np.reshape(npimages_train,(1604,75,75,2))
print npimages_train.shape

(1604, 75, 75, 2)


In [18]:
# number of output classes
nb_classes = 2
# number of epochs to train
nb_epoch = 20

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

X_train, X_test, y_train, y_test = train_test_split(npimages_train, y, test_size=0.2, random_state=4)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print X_train.shape

(1283, 75, 75, 2)


In [44]:
model = Sequential()
model.add(Conv2D(3, (3, 3), input_shape=(75, 75, 2)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(3, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(3,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(3, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.2))
model.add(Dense(2))

# model.add(Convolution2D(10,3,3, border_mode='same'))
# model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 73, 73, 3)         57        
_________________________________________________________________
activation_59 (Activation)   (None, 73, 73, 3)         0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 71, 71, 3)         84        
_________________________________________________________________
activation_60 (Activation)   (None, 71, 71, 3)         0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 35, 35, 3)         0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 33, 33, 3)         84        
_________________________________________________________________
activation_61 (Activation)   (None, 33, 33, 3)         0         
__________

In [45]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
 height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=32)
test_generator = test_gen.flow(X_test, Y_test, batch_size=32)

In [47]:
model.fit_generator(train_generator, steps_per_epoch=X_train.shape[0]//32, epochs=800, 
 validation_data=test_generator, validation_steps=X_test.shape[0]//32)

Epoch 1/800
40/40 [==============================] - 1s 24ms/step - loss: 0.3560 - acc: 0.8312 - val_loss: 0.3165 - val_acc: 0.8625
Epoch 2/800
40/40 [==============================] - 1s 24ms/step - loss: 0.3400 - acc: 0.8452 - val_loss: 0.3383 - val_acc: 0.8313
Epoch 3/800
40/40 [==============================] - 1s 22ms/step - loss: 0.3553 - acc: 0.8183 - val_loss: 0.2840 - val_acc: 0.8875
Epoch 4/800
40/40 [==============================] - 1s 22ms/step - loss: 0.3223 - acc: 0.8593 - val_loss: 0.3391 - val_acc: 0.8531
Epoch 5/800
40/40 [==============================] - 1s 22ms/step - loss: 0.3976 - acc: 0.8261 - val_loss: 0.3846 - val_acc: 0.8281
Epoch 6/800
40/40 [==============================] - 1s 25ms/step - loss: 0.3939 - acc: 0.7941 - val_loss: 0.3268 - val_acc: 0.8469
Epoch 7/800
40/40 [==============================] - 1s 25ms/step - loss: 0.3306 - acc: 0.8421 - val_loss: 0.2700 - val_acc: 0.8688
Epoch 8/800
40/40 [==============================] - 1s 23ms/step - loss: 0.

40/40 [==============================] - 1s 20ms/step - loss: 0.2931 - acc: 0.8621 - val_loss: 0.3422 - val_acc: 0.8469
Epoch 63/800
40/40 [==============================] - 1s 22ms/step - loss: 0.2951 - acc: 0.8733 - val_loss: 0.2470 - val_acc: 0.8719
Epoch 64/800
40/40 [==============================] - 1s 20ms/step - loss: 0.2840 - acc: 0.8773 - val_loss: 0.3542 - val_acc: 0.8375
Epoch 65/800
40/40 [==============================] - 1s 22ms/step - loss: 0.2829 - acc: 0.8737 - val_loss: 0.2915 - val_acc: 0.8719
Epoch 66/800
40/40 [==============================] - 1s 20ms/step - loss: 0.2621 - acc: 0.8874 - val_loss: 0.2549 - val_acc: 0.8781
Epoch 67/800
40/40 [==============================] - 1s 20ms/step - loss: 0.2618 - acc: 0.8820 - val_loss: 0.2636 - val_acc: 0.8812
Epoch 68/800
40/40 [==============================] - 1s 20ms/step - loss: 0.2550 - acc: 0.8843 - val_loss: 0.2719 - val_acc: 0.8875
Epoch 69/800
40/40 [==============================] - 1s 20ms/step - loss: 0.2678 

40/40 [==============================] - 1s 18ms/step - loss: 0.2405 - acc: 0.8929 - val_loss: 0.2659 - val_acc: 0.8562
Epoch 124/800
40/40 [==============================] - 1s 18ms/step - loss: 0.2449 - acc: 0.8874 - val_loss: 0.2496 - val_acc: 0.8875
Epoch 125/800
40/40 [==============================] - 1s 25ms/step - loss: 0.2329 - acc: 0.8976 - val_loss: 0.2463 - val_acc: 0.8906
Epoch 126/800
40/40 [==============================] - 1s 24ms/step - loss: 0.2372 - acc: 0.8929 - val_loss: 0.2385 - val_acc: 0.9000
Epoch 127/800
40/40 [==============================] - 1s 24ms/step - loss: 0.2490 - acc: 0.8840 - val_loss: 0.3017 - val_acc: 0.8688
Epoch 128/800
40/40 [==============================] - 1s 25ms/step - loss: 0.2669 - acc: 0.8777 - val_loss: 0.3049 - val_acc: 0.8594
Epoch 129/800
40/40 [==============================] - 1s 23ms/step - loss: 0.2580 - acc: 0.8797 - val_loss: 0.2613 - val_acc: 0.8906
Epoch 130/800
40/40 [==============================] - 1s 25ms/step - loss: 

40/40 [==============================] - 1s 20ms/step - loss: 0.2235 - acc: 0.8988 - val_loss: 0.2654 - val_acc: 0.8906
Epoch 185/800
40/40 [==============================] - 1s 19ms/step - loss: 0.2365 - acc: 0.8929 - val_loss: 0.2470 - val_acc: 0.9000
Epoch 186/800
40/40 [==============================] - 1s 22ms/step - loss: 0.2466 - acc: 0.8945 - val_loss: 0.3214 - val_acc: 0.8750
Epoch 187/800
40/40 [==============================] - 1s 23ms/step - loss: 0.2400 - acc: 0.8984 - val_loss: 0.2897 - val_acc: 0.8656
Epoch 188/800
40/40 [==============================] - 1s 22ms/step - loss: 0.2538 - acc: 0.8812 - val_loss: 0.2475 - val_acc: 0.9125
Epoch 189/800
40/40 [==============================] - 1s 22ms/step - loss: 0.2346 - acc: 0.8960 - val_loss: 0.2686 - val_acc: 0.8906
Epoch 190/800
40/40 [==============================] - 1s 19ms/step - loss: 0.2599 - acc: 0.8855 - val_loss: 0.2785 - val_acc: 0.8719
Epoch 191/800
40/40 [==============================] - 1s 20ms/step - loss: 

40/40 [==============================] - 1s 20ms/step - loss: 0.2316 - acc: 0.8965 - val_loss: 0.2487 - val_acc: 0.8781
Epoch 246/800
40/40 [==============================] - 1s 18ms/step - loss: 0.2499 - acc: 0.8890 - val_loss: 0.2498 - val_acc: 0.8875
Epoch 247/800
40/40 [==============================] - 1s 19ms/step - loss: 0.2266 - acc: 0.8922 - val_loss: 0.2692 - val_acc: 0.8812
Epoch 248/800
40/40 [==============================] - 1s 24ms/step - loss: 0.2403 - acc: 0.8758 - val_loss: 0.3111 - val_acc: 0.8469
Epoch 249/800
40/40 [==============================] - 1s 25ms/step - loss: 0.2197 - acc: 0.8992 - val_loss: 0.2594 - val_acc: 0.8844
Epoch 250/800
40/40 [==============================] - 1s 25ms/step - loss: 0.2492 - acc: 0.8821 - val_loss: 0.2655 - val_acc: 0.8781
Epoch 251/800
40/40 [==============================] - 1s 24ms/step - loss: 0.2225 - acc: 0.8945 - val_loss: 0.3114 - val_acc: 0.8750
Epoch 252/800
40/40 [==============================] - 1s 25ms/step - loss: 

...

40/40 [==============================] - 1s 20ms/step - loss: 0.2103 - acc: 0.9070 - val_loss: 0.2514 - val_acc: 0.8812
Epoch 551/800
40/40 [==============================] - 1s 19ms/step - loss: 0.1960 - acc: 0.9047 - val_loss: 0.2538 - val_acc: 0.8750
Epoch 552/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1850 - acc: 0.9250 - val_loss: 0.2910 - val_acc: 0.8594
Epoch 553/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1681 - acc: 0.9265 - val_loss: 0.2864 - val_acc: 0.8938
Epoch 554/800
40/40 [==============================] - 1s 19ms/step - loss: 0.2803 - acc: 0.8847 - val_loss: 0.3150 - val_acc: 0.8344
Epoch 555/800
40/40 [==============================] - 1s 20ms/step - loss: 0.2520 - acc: 0.8813 - val_loss: 0.2145 - val_acc: 0.8844
Epoch 556/800
40/40 [==============================] - 1s 22ms/step - loss: 0.1908 - acc: 0.9194 - val_loss: 0.2385 - val_acc: 0.8938
Epoch 557/800
40/40 [==============================] - 1s 22ms/step - loss: 

40/40 [==============================] - 1s 19ms/step - loss: 0.1710 - acc: 0.9265 - val_loss: 0.2867 - val_acc: 0.8719
Epoch 612/800
40/40 [==============================] - 1s 19ms/step - loss: 0.2129 - acc: 0.9231 - val_loss: 0.2369 - val_acc: 0.9031
Epoch 613/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1636 - acc: 0.9281 - val_loss: 0.2796 - val_acc: 0.8750
Epoch 614/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1821 - acc: 0.9242 - val_loss: 0.3233 - val_acc: 0.8531
Epoch 615/800
40/40 [==============================] - 1s 18ms/step - loss: 0.1929 - acc: 0.9171 - val_loss: 0.2932 - val_acc: 0.9000
Epoch 616/800
40/40 [==============================] - 1s 19ms/step - loss: 0.1534 - acc: 0.9332 - val_loss: 0.3504 - val_acc: 0.8875
Epoch 617/800
40/40 [==============================] - 1s 24ms/step - loss: 0.1717 - acc: 0.9382 - val_loss: 0.2826 - val_acc: 0.8812
Epoch 618/800
40/40 [==============================] - 1s 25ms/step - loss: 

40/40 [==============================] - 1s 20ms/step - loss: 0.1999 - acc: 0.9265 - val_loss: 0.3484 - val_acc: 0.8406
Epoch 673/800
40/40 [==============================] - 1s 19ms/step - loss: 0.1861 - acc: 0.9242 - val_loss: 0.4584 - val_acc: 0.8750
Epoch 674/800
40/40 [==============================] - 1s 20ms/step - loss: 0.2599 - acc: 0.8883 - val_loss: 0.2320 - val_acc: 0.8906
Epoch 675/800
40/40 [==============================] - 1s 19ms/step - loss: 0.1652 - acc: 0.9359 - val_loss: 0.3115 - val_acc: 0.8688
Epoch 676/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1488 - acc: 0.9414 - val_loss: 0.2835 - val_acc: 0.8969
Epoch 677/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1585 - acc: 0.9305 - val_loss: 0.3134 - val_acc: 0.8719
Epoch 678/800
40/40 [==============================] - 1s 22ms/step - loss: 0.1444 - acc: 0.9406 - val_loss: 0.3284 - val_acc: 0.8812
Epoch 679/800
40/40 [==============================] - 1s 22ms/step - loss: 

40/40 [==============================] - 1s 20ms/step - loss: 0.1829 - acc: 0.9137 - val_loss: 0.2770 - val_acc: 0.8938
Epoch 734/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1680 - acc: 0.9336 - val_loss: 0.2636 - val_acc: 0.9062
Epoch 735/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1696 - acc: 0.9203 - val_loss: 0.2824 - val_acc: 0.8844
Epoch 736/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1682 - acc: 0.9328 - val_loss: 0.3597 - val_acc: 0.8844
Epoch 737/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1722 - acc: 0.9320 - val_loss: 0.2633 - val_acc: 0.8906
Epoch 738/800
40/40 [==============================] - 1s 17ms/step - loss: 0.1765 - acc: 0.9289 - val_loss: 0.2613 - val_acc: 0.8969
Epoch 739/800
40/40 [==============================] - 1s 19ms/step - loss: 0.1745 - acc: 0.9290 - val_loss: 0.3137 - val_acc: 0.9031
Epoch 740/800
40/40 [==============================] - 1s 24ms/step - loss: 

40/40 [==============================] - 1s 20ms/step - loss: 0.1618 - acc: 0.9180 - val_loss: 0.2424 - val_acc: 0.9031
Epoch 795/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1305 - acc: 0.9461 - val_loss: 0.2259 - val_acc: 0.8938
Epoch 796/800
40/40 [==============================] - 1s 19ms/step - loss: 0.1577 - acc: 0.9343 - val_loss: 0.2735 - val_acc: 0.9094
Epoch 797/800
40/40 [==============================] - 1s 19ms/step - loss: 0.1387 - acc: 0.9499 - val_loss: 0.2656 - val_acc: 0.8812
Epoch 798/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1585 - acc: 0.9238 - val_loss: 0.2746 - val_acc: 0.8875
Epoch 799/800
40/40 [==============================] - 1s 20ms/step - loss: 0.1339 - acc: 0.9414 - val_loss: 0.2822 - val_acc: 0.9000
Epoch 800/800
40/40 [==============================] - 1s 22ms/step - loss: 0.1488 - acc: 0.9301 - val_loss: 0.2880 - val_acc: 0.8781


In [53]:
band1_test = df_test['band_1'].values
band2_test = df_test['band_2'].values
inc_angle_test = df_test['inc_angle'].values
ids = df_test['id'].values

print band1_test.shape
print band2_test.shape
print inc_angle_test.shape

(8424,)
(8424,)
(8424,)


In [54]:
images_test=[]
for cnt,img in enumerate(band1_test):
    images_test.append([img,band2_test[cnt]])
npimages_test = np.array(images_test)
npimages_test=np.reshape(npimages_test,(8424,75,75,2))
print npimages_test.shape

(8424, 75, 75, 2)


In [55]:
predictions = model.predict_classes(npimages_test)

sub = pd.DataFrame({'Actual': ids, 'Predictions': predictions})
print sub

        Actual  Predictions
0     5941774d            0
1     4023181e            1
2     b20200e4            0
3     e7f018bb            1
4     4371c8c3            0
5     a8d9b1fd            0
6     29e7727e            0
7     92a51ffb            1
8     c769ac97            0
9     aee0547d            0
10    565b28ac            0
11    e04e9775            0
12    8e8161d1            0
13    4cf4d256            1
14    139e5324            0
15    f156976f            0
16    68a117cc            0
17    d9aa7a56            0
18    9005b143            0
19    5f6d3988            1
20    9ad70954            0
21    b9087b9e            1
22    a39a1427            0
23    82fbe8ed            0
24    1fae4879            0
25    6dd8f13d            0
26    bbad5958            0
27    54527583            0
28    be8fa29c            0
29    81a3328f            1
...        ...          ...
8394  8ae30ce6            0
8395  de27ed88            0
8396  66d5196f            0
8397  d85f1858      